In [2]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_document = loader.load()
text_document

[Document(metadata={'source': 'speech.txt'}, page_content='It is truly an honor to stand before you today and talk about something that has shaped the greatest minds, the strongest leaders, and the most inspiring individuals throughout history—perseverance.\n\nLife is full of challenges. At some point, we all face setbacks, failures, and moments where we feel like giving up. But what separates those who succeed from those who don’t is not talent, not luck, but the ability to keep going even when things get tough.\n\nThink about some of the world’s most successful people. Thomas Edison failed over 1,000 times before he finally invented the lightbulb. J.K. Rowling was rejected by 12 publishers before Harry Potter became a global phenomenon. And Michael Jordan, one of the greatest basketball players of all time, was cut from his high school team before he became a legend.\n\nWhat do they all have in common? They refused to give up. They believed in their dreams and kept pushing forward, e

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GEMINI_KEY']  = os.getenv('GEMINI_KEY')



In [8]:
## web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/'),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_=('post-title', 'post_content','post-headers')
                       )),)

text_document = loader.load()
text_document

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n      LLM Powered Autonomous Agents\n    ')]

In [9]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('next-gen-js-summary.pdf')
text_document = loader.load()
text_document


[Document(metadata={'producer': 'Mac OS X 10.13.3 Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20180224150452Z00'00'", 'title': 'Untitled.pages', 'moddate': "D:20180224150452Z00'00'", 'source': 'next-gen-js-summary.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='In this module, I\xa0provided a brief introduction into some \ncore next-gen JavaScript features, of course focusing on \nthe ones you\'ll see the most in this course. Here\'s a quick \nsummary!\nlet &\xa0const \nRead more about\xa0let\xa0:\xa0https://developer.mozilla.org/en-US/\ndocs/Web/JavaScript/Reference/Statements/let\nRead more about\xa0const\xa0:\xa0https://developer.mozilla.org/en-\nUS/docs/Web/JavaScript/Reference/Statements/const\nlet\xa0 and\xa0const\xa0 basically replace\xa0var\xa0. You use\xa0let\xa0 \ninstead of\xa0var\xa0 and\xa0const\xa0 instead of\xa0var\xa0 if you plan on \nnever re-assigning this "variable"\xa0(eﬀectively turning it into a \nconstant therefore).\nES6 Arrow

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter =RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents= text_splitter.split_documents(text_document)
documents[:5]

[Document(metadata={'producer': 'Mac OS X 10.13.3 Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20180224150452Z00'00'", 'title': 'Untitled.pages', 'moddate': "D:20180224150452Z00'00'", 'source': 'next-gen-js-summary.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='In this module, I\xa0provided a brief introduction into some \ncore next-gen JavaScript features, of course focusing on \nthe ones you\'ll see the most in this course. Here\'s a quick \nsummary!\nlet &\xa0const \nRead more about\xa0let\xa0:\xa0https://developer.mozilla.org/en-US/\ndocs/Web/JavaScript/Reference/Statements/let\nRead more about\xa0const\xa0:\xa0https://developer.mozilla.org/en-\nUS/docs/Web/JavaScript/Reference/Statements/const\nlet\xa0 and\xa0const\xa0 basically replace\xa0var\xa0. You use\xa0let\xa0 \ninstead of\xa0var\xa0 and\xa0const\xa0 instead of\xa0var\xa0 if you plan on \nnever re-assigning this "variable"\xa0(eﬀectively turning it into a \nconstant therefore).\nES6 Arrow

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Correct embedding model
    google_api_key=os.getenv('GEMINI_KEY')
)
db = Chroma.from_documents(documents[:15], embedding_function)

In [18]:
##vector databases 

query='what is ES6 Arrow Functions '
result = db.similarity_search(query)
result[0].page_content

'In this module, I\xa0provided a brief introduction into some \ncore next-gen JavaScript features, of course focusing on \nthe ones you\'ll see the most in this course. Here\'s a quick \nsummary!\nlet &\xa0const \nRead more about\xa0let\xa0:\xa0https://developer.mozilla.org/en-US/\ndocs/Web/JavaScript/Reference/Statements/let\nRead more about\xa0const\xa0:\xa0https://developer.mozilla.org/en-\nUS/docs/Web/JavaScript/Reference/Statements/const\nlet\xa0 and\xa0const\xa0 basically replace\xa0var\xa0. You use\xa0let\xa0 \ninstead of\xa0var\xa0 and\xa0const\xa0 instead of\xa0var\xa0 if you plan on \nnever re-assigning this "variable"\xa0(eﬀectively turning it into a \nconstant therefore).\nES6 Arrow Functions \nRead more:\xa0https://developer.mozilla.org/en-US/docs/Web/\nJavaScript/Reference/Functions/Arrow_functions\nArrow functions are a diﬀerent way of creating functions in \nJavaScript. Besides a shorter syntax, they oﬀer advantages \nwhen it comes to keeping the scope of the\xa0this\xa

In [19]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents[:15], embedding_function)
query = "what is an arrow function"
result = db.similarity_search(query)
result[0].page_content

"JavaScript. Besides a shorter syntax, they oﬀer advantages \nwhen it comes to keeping the scope of the\xa0this\xa0 keyword \n(see\xa0here).\nArrow function syntax may look strange but it's actually \nsimple.\n.f u n c t i o n callMe(name) { \n.    c o n s o l e . l o g ( n a m e ) ;\n.}"

In [21]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model='gemma')
llm


OllamaLLM(model='gemma')

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on only on the privoided context.
Think step by step before providing the answer.
<context>
{context}
</context>
Question: {input}                                          
"""
)

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document = create_stuff_documents_chain(llm, prompt)

In [26]:
retriever  =  db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x74a9e97a96c0>, search_kwargs={})

In [29]:
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document)


In [33]:
resposne = retriever_chain.invoke({'input':"ES6 Arrow Functions"})


In [35]:
resposne['answer']

'The provided text discusses ES6 Arrow Functions and their advantages in keeping the scope of the `this` keyword. \n\n**Key points:**\n\n- Arrow functions are a different way of creating functions in JavaScript.\n- They offer advantages when it comes to maintaining the scope of the `this` keyword.\n- Arrow function syntax is simple despite its seemingly strange appearance.\n\n**Therefore, the answer to the question is:** ES6 Arrow Functions provide a way to define functions where the `this` keyword retains its original scope, making them suitable for situations where the context of `this` is important.'